In [0]:
import numpy as np
import pandas as pd
import ast

In [0]:
authors = pd.read_csv('authors.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
def stringtolist(str):
    return ast.literal_eval(str)
def len_list(author_id):
  return len(author_id)

In [0]:
def interest_set(interests):
    return list(set([y for x in interests for y in x]))

In [0]:
authors.head()

,id,affiliation,published-papers,citations,h-index,p-equal-a,p-unequal-a,interests
0,1101155,"Northeastern University, Boston, MA, USA",1,0,0,0.0000,0.0000,"['social network', 'social network-enabled EAP..."
1,1377286,"Department of Computer Science, University of ...",2,14,0,4.5000,1.8750,"['D scheme', 'split task', 'mixed criticality ..."
2,1613605,NaN,1,0,0,0.0000,0.0000,"['present value', 'expected value', 'Random Ra..."
3,629007,NaN,1,0,0,0.0000,0.0000,"['ride comfort', 'different vehicle speed', 'v..."
4,1390253,"United Institute of Informatics Problems, Nati...",6,5,2,1.3333,0.3611,"['spindle head', 'decision support system', 'm..."


In [0]:
def process(author_id):
    listrey = ast.literal_eval(author_id)
    pps,cits,hidx,pea,puea,aff_authors,interests=0,0,0,0,0,0,0
    interests,affs = [],[]
    for i in listrey:
        i = int(i)
        pps += authors.loc[i]['published-papers']
        cits += authors.loc[i]['citations']
        hidx += authors.loc[i]['h-index']
        pea += authors.loc[i]['p-equal-a']
        puea += authors.loc[i]['p-unequal-a']
        interests.append(authors.loc[i]['interests'])
        if (pd.isna(authors.loc[i]['affiliation']))==False:
          affs.append(authors.loc[i]['affiliation'])
    aff_authors = len(affs)
    return [pps,cits,hidx,pea,puea,aff_authors,interests]

In [0]:
authors.index = authors.id
train['new_data'] = train['author_id'].apply(process)
new_data = pd.DataFrame(train['new_data'].values.tolist(), columns=['published_papers','authors_citations','h_index','pequala','punequala','aff_authors','interests'])
train = pd.concat((train,new_data),1)
train.drop('new_data',1,inplace=True)
train['author_id'] = train['author_id'].apply(stringtolist)
train['authors'] = train['author_id'].apply(len_list)

In [0]:
def no_of_interests(data):
  length = 0
  for i in data:
    length += len(i.split(','))
  return length

In [0]:
train['interests'] = train['interests'].apply(no_of_interests)

In [0]:
authors_interests = 0
interests_list = []
for i in list(authors['interests']):
  singlelist = ast.literal_eval(i)
  for j in singlelist:
    interests_list.append(j)
authors_interests = len(set(interests_list))

In [0]:
train['interests_pct'] = (train['interests']/authors_interests)*100

In [0]:
train.head()

,author_id,year,venue_id,citations,id,published_papers,authors_citations,h_index,pequala,punequala,aff_authors,interests,authors,interests_pct
0,"[302080, 254034, 1650574, 1688060, 356214, 144...",2011.0,15632,0,162253,23,12,7,1.5929,2.0285,2,81,9,0.003248
1,[471182],2010.0,15752,0,42448,1,0,0,0.0000,0.0000,1,10,1,0.000401
2,[1252574],2012.0,19098,0,226093,27,48,4,39.6667,43.7222,1,10,1,0.000401
3,"[1700521, 1634966, 1560119, 1588537, 1657765]",2012.0,20323,29,281211,38,299,9,151.1667,120.1194,2,50,5,0.002005
4,"[750555, 1125222]",2010.0,12539,0,8973,4,0,0,0.0000,0.0000,0,20,2,0.000802


Thought it is better to save the processed dataframe in a new csv file as it is taking much time for preprocessing.

In [0]:
train2 = train
train2.to_csv('train2.csv')

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

y = train2['citations']
X = train2.drop(['author_id','id','citations','interests','year','venue_id'],1)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X.iloc[:,:-1] = sc.fit_transform(X.iloc[:,:-1])

In [0]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2)

In [0]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(r2_score(y_test,y_pred))
import xgboost

# Let's try XGboost algorithm to see if we can get better results
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train)
print(r2_score(y_test, xgb.predict(X_test)))

0.11367184196612101
[18:40:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.15408548030149916


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               1152      
_________________________________________________________________
dense_11 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 34,433
Trainable params: 34,433
Non-trainable params: 0
_________________________________________________________________


In [0]:
NN_model.fit(X_train, y_train, epochs=50, batch_size=256, validation_split = 0.2)

In [0]:
from keras.layers import Dense, Activation, LeakyReLU,Dropout
from keras.models import Sequential
from keras.layers import LeakyReLU
from keras import regularizers
# we still need to clear a graph though

model = Sequential()  # it is a feed-forward network without loops like in RNN
model.add(Dense(16, input_shape=(8,)))  # the first layer must specify the input shape (replacing placeholders)

model.add(Dense(4))

model.add(Dense(1))

from keras.callbacks import EarlyStopping, ModelCheckpoint
model.compile(
      loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(
    X_train, 
    y_train,
    batch_size=64, 
    epochs=20,
    validation_data=(X_test, y_test),
    verbose=1
)

In [0]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.5)
reg.fit(X_train,y_train)
y_predreg = reg.predict(X_test)
r2_score(y_test,y_predreg)

0.11368115648873267

In [0]:
test['new_data'] = test['author_id'].apply(process)
new_data = pd.DataFrame(test['new_data'].values.tolist(), columns=['published_papers','authors_citations','h_index','pequala','punequala','aff_authors','interests'])
test = pd.concat((test,new_data),1)
test.drop('new_data',1,inplace=True)
test['author_id'] = test['author_id'].apply(stringtolist)
test['authors'] = test['author_id'].apply(len_list)

In [0]:
test['interests'] = test['interests'].apply(no_of_interests)
test['interests_pct'] = (test['interests']/authors_interests)*100

In [0]:
test2 = test
test2.to_csv('test2.csv')

In [0]:
test = test.iloc[:,4:]
test.drop(['interests'],1,inplace = True)

In [0]:
test.iloc[:,:-1] = sc.transform(test.iloc[:,:-1])

In [0]:
sub2 = {'id':test2['id']}
citations = lr.predict(test)
sub2['citations'] = citations
sub2 = pd.DataFrame(sub2)
sub2.to_csv('sub2.csv',index=False)

In [0]:
subxgb2 = {'id':test2['id']}
citations = xgb.predict(test)
subxgb2['citations'] = citations
subxgb2 = pd.DataFrame(subxgb2)
subxgb2.to_csv('subxgb2.csv',index=False)

In [0]:
train2 = pd.read_csv('train2.csv')

In [0]:
train2['h_index'] = train2['h_index']/train2['authors']
train2['pequala'] = train2['pequala']/train2['authors']
train2['punequala'] = train2['punequala']/train2['authors']

In [0]:
dfDummies = pd.get_dummies(train2['year'], prefix = 'category')

In [0]:
train2 = pd.concat([train2,dfDummies],1)

In [0]:
train2.head()

,Unnamed: 0,author_id,year,venue_id,citations,id,published_papers,authors_citations,h_index,pequala,punequala,aff_authors,interests,authors,interests_pct,category_2010.0,category_2011.0,category_2012.0,category_2013.0,category_2014.0
0,0,"['302080', '254034', '1650574', '1688060', '35...",2011.0,15632,0,162253,23,12,0.777778,0.176989,0.225389,2,81,9,0.003248,0,1,0,0,0
1,1,['471182'],2010.0,15752,0,42448,1,0,0.000000,0.000000,0.000000,1,10,1,0.000401,1,0,0,0,0
2,2,['1252574'],2012.0,19098,0,226093,27,48,4.000000,39.666700,43.722200,1,10,1,0.000401,0,0,1,0,0
3,3,"['1700521', '1634966', '1560119', '1588537', '...",2012.0,20323,29,281211,38,299,1.800000,30.233340,24.023880,2,50,5,0.002005,0,0,1,0,0
4,4,"['750555', '1125222']",2010.0,12539,0,8973,4,0,0.000000,0.000000,0.000000,0,20,2,0.000802,1,0,0,0,0


In [0]:
test2 = pd.read_csv('test2.csv')

In [0]:
test2['h_index'] = test2['h_index']/test2['authors']
test2['pequala'] = test2['pequala']/test2['authors']
test2['punequala'] = test2['punequala']/test2['authors']
dfDummies = pd.get_dummies(test2['year'], prefix = 'category')
test2 = pd.concat([test2,dfDummies],1)

In [0]:
test2.head()

,Unnamed: 0,author_id,year,venue_id,id,published_papers,authors_citations,h_index,pequala,punequala,aff_authors,interests,authors,interests_pct,category_2010.0,category_2011.0,category_2012.0,category_2013.0,category_2014.0
0,0,['1138048'],2010.0,22698,77152,1,0,0.00,0.00000,0.000000,1,8,1,0.000321,1,0,0,0,0
1,1,"['1111668', '1154294']",2012.0,1037,253904,15,15,1.50,1.90000,3.160550,2,20,2,0.000802,0,0,1,0,0
2,2,"['1505756', '1382981']",2010.0,22438,46386,2,4,1.00,0.66665,0.472200,0,8,2,0.000321,1,0,0,0,0
3,3,"['634857', '542921']",2010.0,21723,19572,45,215,4.00,32.18335,46.817200,2,20,2,0.000802,1,0,0,0,0
4,4,"['1709244', '1240251', '490730', '1525100']",2012.0,22453,182480,32,43,1.75,3.46250,3.955625,3,40,4,0.001604,0,0,1,0,0


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

y = train2['citations']
X = train2.drop(['Unnamed: 0','author_id','id','citations','interests','year','venue_id'],1)
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2)

In [0]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.5)
reg.fit(X_train,y_train)
y_predreg = reg.predict(X_test)
r2_score(y_test,y_predreg)

0.11243262481644667

In [0]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(r2_score(y_test,y_pred))
import xgboost

# Let's try XGboost algorithm to see if we can get better results
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train)
print(r2_score(y_test, xgb.predict(X_test)))

0.11254382213669678
[12:28:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.1612371586659409


In [0]:
subxgb3 = {'id':test2['id']}
citations = xgb.predict(test2.drop(['Unnamed: 0','author_id','id','interests','year','venue_id'],1))
subxgb3['citations'] = citations
subxgb3 = pd.DataFrame(subxgb3)
subxgb3.to_csv('subxgb3.csv',index=False)